# Линейная регрессия II

## План занятия:

1. Применение линейной регрессии на реальных данных
2. Нормализация данных
3. Полиномиальная регрессия
4. Переобучение и недообучение
5. Средняя абсолютная функция ошибки

In [ ]:
from regression2_helper import * # Подгружаем функции для визуализации
import numpy as np              # Подгруджаем библиотеку NumPy
np.set_printoptions(formatter={'float': lambda x: format(x, '6.3f')}) # настройка вывода для матриц

# Применение линейной регрессии на реальных данных

Рассмотрим реальный датасет содержащий данные о стоимости жилья в Бостоне.  
Данные были собраны в 1978 году и содержат 506 записей, представляющие собой 14 агрегированных характеристик домов из пригородов Бостона, штат Массачусетс.

1. **CRIM**: Уровень преступности  
2. **ZN**: Доля жилой земли, отведенной под участки более 25000 кв. Футов  
3. **INDUS**: Доля неторговых площадей на город  
4. **CHAS**: Фиктивная переменная реки Чарльз (= 1, если путь ограничивает реку; 0 в противном случае)  
5. **NOX**: Концентрация оксида азота  
6. **RM**: Среднее количество комнат в доме  
7. **AGE**: Доля домов, построенных до 1940 года  
8. **DIS**: Взвешенные расстояния до пяти бостонских центров занятости  
9. **RAD**: Индекс доступности к радиальным магистралям  
10. **TAX**: Ставка налога на полную стоимость имущества за 10000 долларов США  
11. **PTRATIO**: Соотношение учеников и учителей по городам   
12. **B**: $1000(Bk - 0,63)^2$, где $Bk$ - это доля людей афроамериканского происхождения по городам  
13. **LSTAT**: процент населения с более низким статусом чем у жильца  
14. **MEDV**: медианная цена за дом с такими параметрами в 1000$

Наша задача будет предсказать цену домов (MEDV) используя данные характеристики

## Регрессия на двух параметрах 

Для начала построим модель линейной регрессии для предсказания медианной цены домов,   
используя всего два признака: RM и LSTAT.

In [ ]:
#загрузим данные
X_room, X_lstat, y = load_small_data() 

In [ ]:
print_3d_table_with_data(X_room, X_lstat, y)

Рассмотрим зависимости каждого из признаков с ценой.

In [ ]:
plot_rm(X_room, y)

In [ ]:
plot_lstat(X_lstat, y)

Рассмотрим теперь зависимость от двух признаков.

In [ ]:
plot_new_3d_data(X_room, X_lstat, y)

## Функция от двух параметров

Для предсказания цен квартир в Бостоне по данной паре признаков, рассмотрим модель линейной регрессию вида:
\begin{equation*}
\tilde{y} = \theta_0 + \theta_1 x_1 + \theta_2 x_2.
\end{equation*}
$\tilde{y}$ - предсказанная цена  
$x_1$ - Cреднее количество комнат  
$x_2$ - LSTAT %  
$\theta_0, \theta_1, \theta_2$ - параметры модели (веса)

Визуализируем как выглядит эта функция.

In [ ]:
plot_new_3d_data_and_hyp(X_room, X_lstat, y)

### Линейная функция в матричном виде

Давайте вспомним как производить матричное произведение.

\begin{equation*}
A = \begin{pmatrix}
    a_{0, 0} & a_{0, 1} & \ldots & a_{0, j} \ldots a_{0, m} \\
    a_{1, 0} & a_{1, 1} & \ldots & a_{1, j} \ldots a_{1, m} \\
    \cdot &   \cdot  & \cdot &    \cdot \\
    a_{i, 0} & a_{i, 1} & \ldots & a_{i, j} \ldots a_{i, m} \\
    \cdot &   \cdot  & \cdot &    \cdot \\
    a_{n, 0} & a_{N, 1} & \ldots & a_{n, j} \ldots a_{n, m} \\
\end{pmatrix}
\end{equation*}

Размер $A = (n, m)$.


\begin{equation*}
X = \begin{pmatrix}
    x_{0, 0} & x_{0, 1} & \ldots & x_{0, j} \ldots x_{0, k} \\
    x_{1, 0} & x_{1, 1} & \ldots & x_{1, j} \ldots x_{1, k} \\
    \cdot &   \cdot  & \cdot &    \cdot \\
    x_{i, 0} & x_{i, 1} & \ldots & x_{i, j} \ldots x_{i, k} \\
    \cdot &   \cdot  & \cdot &    \cdot \\
    x_{m, 0} & x_{m, 1} & \ldots & x_{m, j} \ldots x_{m, k} \\
\end{pmatrix}
\end{equation*}

Размер $X = (m, k)$.


\begin{equation*}
Y = A \cdot X= \begin{pmatrix}
    \sum_i^m a_{0, i}x_{i, 0} & \ldots & \sum_i^m a_{0, i}x_{i, j} & \ldots & \sum_i^m a_{0, i}x_{i, k} \\
    \sum_i^m a_{1, i}x_{i, 1} & \ldots & \sum_i^m a_{1, i}x_{i, j} & \ldots & \sum_i^m a_{1, i}x_{i, k} \\
    \cdot &   \cdot  & \cdot &    \cdot \\
    \sum_i^m a_{k, i}x_{i, k} & \ldots & \sum_i^m a_{k, i}x_{i, j} & \ldots & \sum_i^m a_{k, i}x_{i, k} \\
    \cdot &   \cdot  & \cdot &    \cdot \\
    \sum_i^m a_{n, i}x_{i, n} & \ldots & \sum_i^m a_{n, i}x_{i, j} & \ldots & \sum_i^m a_{n, i}x_{i, k} \\
\end{pmatrix}
\end{equation*}

Размер $Y = (n, k)$.

И отдельно рассмотрим произведение матрицы и вектора строки на вектор колонку.

#### Произведение векторов

\begin{equation*}
z = \begin{pmatrix}
    z_{0, 0} & z_{0, 1} & \ldots & z_{0, j} \ldots z_{0, m} \\
\end{pmatrix}
\end{equation*}

\begin{equation*}
v = \begin{pmatrix}
    v_{0, 0} \\
    v_{1, 0} \\
    \ldots \\
    v_{j, 0} \\
    \ldots \\
    v_{m, 0}
\end{pmatrix}
\end{equation*}

Часто для удобства $v$ записывают как \begin{equation*}
v = \begin{pmatrix}
    v_{0, 0} &
    v_{1, 0} &
    \ldots &
    v_{j, 0} &
    \ldots &
    v_{m, 0}
\end{pmatrix}^T
\end{equation*}

$z v = \sum_i^m z_{0, i}v_{i, 0}$

#### Произведение матрицу на вектор

\begin{equation*}
A \cdot v = \begin{pmatrix}
    a_{0, 0} & a_{0, 1} & \ldots & a_{0, j} \ldots a_{0, m} \\
    a_{1, 0} & a_{1, 1} & \ldots & a_{1, j} \ldots a_{1, m} \\
    \cdot &   \cdot  & \cdot &    \cdot \\
    a_{i, 0} & a_{i, 1} & \ldots & a_{i, j} \ldots a_{i, m} \\
    \cdot &   \cdot  & \cdot &    \cdot \\
    a_{n, 0} & a_{N, 1} & \ldots & a_{n, j} \ldots a_{n, m} \\
\end{pmatrix}
\begin{pmatrix}
    v_{0, 0} \\
    v_{1, 0} \\
    \ldots \\
    v_{j, 0} \\
    \ldots \\
    v_{m, 0}
\end{pmatrix}
\end{equation*}

\begin{equation*}
A \cdot v = \begin{pmatrix}
    a_{0, 0}v_{0, 0} + a_{0, 1}v_{1, 0} + \ldots + a_{0, j}v_{j, 0} + \ldots +a_{0, m}v_{m, 0} \\
    a_{1, 0}v_{0, 0} + a_{1, 1}v_{1, 0} + \ldots + a_{2, j}v_{j, 0} + \ldots +a_{2, m}v_{m, 0} \\
    \cdots \\
    a_{i, 0}v_{0, 0} + a_{i, 1}v_{1, 0} + \ldots + a_{i, j}v_{j, 0} + \ldots +a_{i, m}v_{m, 0} \\
    \cdots \\
    a_{n, 0}v_{0, 0} + a_{n, 1}v_{1, 0} + \ldots + a_{n, j}v_{j, 0} + \ldots +a_{n, m}v_{m, 0} \\
\end{pmatrix} =
\begin{pmatrix}
    \sum_i^m a_{0, i}v_{i, 0} \\
    \sum_i^m a_{1, i}v_{i, 0} \\
    \cdots \\
    \sum_i^m a_{j, i}v_{i, 0} \\
    \cdots \\
    \sum_i^m a_{n, i}v_{i, 0} \\
\end{pmatrix}
\end{equation*}

#### Как это можно использовать?

Давайте объединим признаки для $i$-того примера в вектор стоку 
\begin{equation*}
x_i = \begin{pmatrix} x_{i, 1} & x_{i, 2}
\end{pmatrix}
\end{equation*}


$x_{i, 1}$ - это среднее количество комнат $i$-го примера.

$x_{i, 2}$ - LSTAT $i$-го примера.

Давайте также объединим коэффициенты в вектор столбец $\theta = \begin{pmatrix}
    \theta_1\\
    \theta_2\\
  \end{pmatrix} =  \begin{pmatrix}\theta_1 & \theta_2 \end{pmatrix}^T$


Объединив признаки и коэффициенты в вектора можем переписать предсказание $i$-того примера в матричном виде:
  \begin{equation*}
\tilde{y_i}= \theta_0 + \begin{pmatrix}x_{i, 1} & x_{i, 2}\end{pmatrix} \begin{pmatrix}
    \theta_1\\
    \theta_2\\
  \end{pmatrix} = \theta_0 + \theta_1 x_{i, 1} + \theta_2 x_{i, 2}.
\end{equation*}

Или переписав использовав матричное умножение:
$\tilde{y}= \theta_0 + x_i \theta$.

Чтобы избавиться от свободного члена $\theta_0$  и записать выражение полностью в матричном виде, сделаем следующий трюк: добавим к вектору $x_i$  единицу:

${X}_i = \begin{pmatrix} 1 & x_{i, 1} & x_{i, 2}\end{pmatrix}$

Тогда $\theta_0$ можно добавить в вектор столбец $\theta$ первым элементом:
\begin{equation*}
\Theta = \begin{pmatrix}\theta_0 &  \theta_1 & \theta_2\end{pmatrix}^T,
\end{equation*}

а вычисление предсказанной цены $\tilde{y}_i$ сведется всего лишь к матричному произведению:
  \begin{equation*}
\tilde{y}_i= \begin{pmatrix} 1 & x_{i, 1} & x_{i, 2}\end{pmatrix} \begin{pmatrix}
    \theta_0\\
    \theta_1\\
    \theta_2\\
  \end{pmatrix} =  1 \cdot \theta_0 + \theta_1 x_{i, 1} + \theta_2 x_{i, 2}.
\end{equation*}

\begin{equation*}
\tilde{y}_i = X_i\Theta.
\end{equation*}

Эта формула позволяет вычислить предсказание для одного конкретного жилья в Бостоне.

Плюсом матричного произведение является то, что мы можем вычислить предсказание стоимости жилья сразу для всех квартир.

Допустим у нас есть $N$ примеров в обучающей выборке. $x_{i,0} = 1$. Тогда весь набор можно записать как:

\begin{equation*}
X= \begin{pmatrix}
1& x_{1,1}& x_{1,2} \\
1& x_{2,1}& x_{2,2} \\
    \cdot\\
    \cdot\\
1& x_{i,1}& x_{i,2} \\
    \cdot\\
    \cdot\\
    \cdot\\
1& x_{N,1}& x_{N,2} \\
\end{pmatrix}=
\begin{pmatrix}
x_{1,0}& x_{1,1}& x_{1,2} \\
x_{2,0}& x_{2,1}& x_{2,2} \\
    \cdot\\
    \cdot\\
x_{i,0}& x_{i,1}& x_{i,2} \\
    \cdot\\
    \cdot\\
    \cdot\\
x_{N,0}& x_{N,1}& x_{N,2} \\
\end{pmatrix}
\end{equation*}

или

\begin{equation*}
X= \begin{pmatrix}
X_0 \\
X_1 \\
    \cdot\\
    \cdot\\
X_i \\
    \cdot\\
    \cdot\\
    \cdot\\
X_N \\
\end{pmatrix}
\end{equation*}

Или в самом коротком виде: $X = \begin{pmatrix} X_1 & X_2 & \ldots & X_N \end{pmatrix}^T$, где $X_i = \begin{pmatrix} x_{i,0} & x_{i,1} & x_{i,2}\end{pmatrix}$.


Тогда для предсказания стоимости жилья всех квартир, запишим $N$ выражений:


\begin{cases}
\tilde{y}_1 = 1\cdot \theta_0 + \theta_1 x_{1,1} + \theta_2 x_{1,2}\\
\tilde{y}_2 = 1\cdot \theta_0 + \theta_1 x_{2,1} + \theta_2 x_{2,2}\\
\cdot\\
\cdot\\
\tilde{y}_i = 1\cdot \theta_0 + \theta_1 x_{i,1} + \theta_2 x_{i,2}\\
\cdot\\
\cdot\\
\cdot\\
\tilde{y}_N = 1\cdot \theta_0 + \theta_1 x_{N,1} + \theta_2 x_{N,2}\\
\end{cases}

 \begin{cases}
\tilde{y}_1 = x_{0,1}\theta_0 + \theta_1 x_{1,1} + \theta_2 x_{1,2}\\
\tilde{y}_2 = x_{1,1}\theta_0 + \theta_1 x_{2,1} + \theta_2 x_{2,2}\\
\cdot\\
\cdot\\
\tilde{y}_i = x_{i,1}\theta_0 + \theta_1 x_{i,1} + \theta_2 x_{i,2}\\
\cdot\\
\cdot\\
\cdot\\
\tilde{y}_N = x_{N,1}\theta_0 + \theta_1 x_{N,1} + \theta_2 x_{N,2}\\
\end{cases}

Или тоже самое можно записать в матричном виде:  
\begin{equation*}
 \begin{pmatrix}
x_{1,0}& x_{1,1}& x_{1,2} \\
x_{2,0}& x_{2,1}& x_{2,2} \\
    \cdot\\
    \cdot\\
x_{i,0}& x_{i,1}& x_{i,2} \\
    \cdot\\
    \cdot\\
    \cdot\\
x_{N,0}& x_{N,1}& x_{N,2} \\
\end{pmatrix}\cdot
\begin{pmatrix}
    \theta_0\\
    \theta_1\\
    \theta_2
  \end{pmatrix}=\begin{pmatrix}
x_{0,1}\theta_0 + \theta_1 x_{1,1} + \theta_2 x_{1,2}\\
x_{1,1}\theta_0 + \theta_1 x_{2,1} + \theta_2 x_{2,2}\\
\cdot\\
\cdot\\
x_{i,1}\theta_0 + \theta_1 x_{i,1} + \theta_2 x_{i,2}\\
\cdot\\
\cdot\\
\cdot\\
x_{N,1}\theta_0 + \theta_1 x_{N,1} + \theta_2 x_{N,2}\\
  \end{pmatrix}=
  \begin{pmatrix}
    \tilde{y}_1\\
    \tilde{y}_2\\
    \cdot\\
    \cdot\\
    \tilde{y}_i\\
    \cdot\\
    \cdot\\
    \cdot\\
    \tilde{y}_N\\
  \end{pmatrix}
\end{equation*}


\begin{equation*}
\tilde{y} = X\Theta.
\end{equation*}

$x_{i, 0} = 1$  

Для начала составим матрицу $X$ из наших признаков.

In [ ]:
def create_data(X1, X2):
    X_ones = np.ones_like(X1)
    return np.column_stack([X_ones, X1, X2])

X = create_data(X_room, X_lstat)
print(X)

Зададим начальные параметры весов случайными числами от 0 до 1:

In [ ]:
Theta = np.random.random_sample(size=(3,))
print(Theta)

Реализуем линейную функцию от наших параметров и данных (функция делает $N$ предсказаний по параметрам $\Theta$).

In [ ]:
def linear_function(X, Theta):
    return np.dot(X, Theta) #X @ Theta

y_pred = linear_function(X, Theta)

In [ ]:
print(len(X))

In [ ]:
print(len(y_pred))

### Функция ошибки

Теперь, когда мы можем сделать предсказание для всех квартир из выборки, определим функцию ошибки от параметров $\Theta$ :

\begin{equation*}
Loss(\Theta) = \dfrac{1}{N}\sum_{i=1}^{N}{(\tilde{y_i} - y_i)^2}= \dfrac{1}{N} \sum_{i=1}^{N}{(X_i\Theta - y_i)^2}
\end{equation*}

Где $N$ - это количество квартир, $y_i$ - реальная цена квартиры, $\tilde{y_i}$ - предсказанная цена для $i$-oй квартиры.   

Реализуем функцию ошибку наших предсказаний.

In [ ]:
def MSE_Loss(X, Theta, y_true):
    y_pred = linear_function(X, Theta)
    return np.mean((y_pred - y_true)**2)

print(MSE_Loss(X, Theta, y))

Мы хотим минимизировать данную ошибку, для этого нам надо правильно подобрать параметры $\Theta$.  

Для нахождения параметров $\Theta$, также используем градиентный спуск.

### Градиент

Так как у нас 3 параметра $\theta_0, \theta_1, \theta_2$, которые мы хотим найти, надо брать частную производную по каждому из них.  
Посчитаем частную производную по каждому параметру от нашей функции ошибки:

\begin{equation*}
\dfrac{\partial Loss(\Theta)}{\partial \theta_j} = \dfrac{\partial}{\partial \theta_j}(\dfrac{1}{N} \sum_{i=1}^{N}{(X_i\Theta - y_i)^2})
\end{equation*}

\begin{equation*}
\dfrac{\partial Loss(\Theta)}{\partial \theta_j} = \dfrac{2}{N} \sum_{i=1}^{N}(X_i\Theta - y_i)\dfrac{\partial (X_i\Theta  - y_i)}{\partial \theta_j}
\end{equation*}

\begin{equation*}
\dfrac{\partial Loss(\Theta)}{\partial \theta_j} = \dfrac{2}{N} \sum_{i=1}^{N}(X_i\Theta - y_i)\dfrac{\partial (\theta_0x_{i, 0} + \theta_1 x_{i,1} + \theta_2 x_{i,2}  - y_i)}{\partial \theta_j}
\end{equation*}

\begin{equation*}
\dfrac{\partial Loss(\Theta)}{\partial \theta_j} = \dfrac{2}{N} \sum_{i=1}^{N}(X_i\Theta - y_i) x_{ij}
\end{equation*}
  
$j = 0,1,2$.  
$y_i$ - это реальная цена на квартиру, она константа, производная от константы равна нулю.  


$\tilde{y_i} = X_i\Theta = \theta_0x_{i,0} + \theta_1x_{i,1} +  \theta_2x_{i,2}$ - наше предсказание.   


Частная производная
$\dfrac{\partial \tilde{y_i}}{\partial \theta_j} = x_{i,j}$.  
$x_{i,0} = 1$.  

Взяв частную производную по каждому параметру  $\theta_j$, получим вектор градиент функции ошибки:

\begin{equation*}
\nabla Loss(\Theta) =
   \begin{bmatrix}
   \dfrac{2}{N} \sum_{i=1}^{N} (X_i\Theta - y_i)x_{i,0}\\
   \dfrac{2}{N} \sum_{i=1}^{N} (X_i\Theta - y_i)x_{i,1}\\
   \dfrac{2}{N} \sum_{i=1}^{N} (X_i\Theta - y_i)x_{i,2}\\
   \end{bmatrix}
\end{equation*}


Приравняв каждую частную производную (элемент вектора градиента) к нулю и решив систему уравнений, сможем найти наилучшие параметры $\Theta$.
   
 \begin{cases}
   \dfrac{2}{N} \sum_{i=1}^{N} (X_i\Theta - y_i)x_{i,0} = 0\\
   \dfrac{2}{N} \sum_{i=1}^{N} (X_i\Theta - y_i)x_{i,1} = 0\\
   \dfrac{2}{N} \sum_{i=1}^{N} (X_i\Theta - y_i)x_{i,2} = 0\\
 \end{cases}

   Но делать это не целесообразно. Для решения данной системы в компьютере потребуется построить матричное уравнение. А в процессе решения потребуется найти обратную матрицу (сложность $O(n^3)$). Данная операция является очень медленной, даже на современных компьютерах. В данном примере у нас всего 2 входных параметра (RM и LSTAT) и $N=506$ значений $X$. Вычисления обратной матрицы для нашего примера займет микросекунды. Но в реальных приложениях обычно бывает и по десяткам тысяч входных параметров и сотни миллионов значений. Нахождения обратной матриц для таких задач займет несравнимо много времени по сравнению с градиентным спуском. Поэтому в промышленности применяется именно градиентный спуск. 

Реализуем подсчет градиента.

In [ ]:
def gradient_function(Theta, X, y_true):
    grad = np.zeros_like(Theta)
    y_pred = linear_function(X, Theta)
    for j in range(Theta.shape[0]):       
        grad[j] = 2*np.mean((y_pred - y_true)* X[:,j])
    return grad

gard = gradient_function(Theta, X, y)
print(gard)

## Градиентный спуск

Для нашего примера с 3 коэффициентами, алгоритм градиентного спуска будет выглядеть следующим образом:  
$\theta_{j_{new}} = \theta_j - \alpha \dfrac{\partial Loss(\Theta)}{\partial \theta_j}$  
$\theta_{j_{new}} = \theta_j - \alpha  \dfrac{2}{N} \sum_{i=1}^{N} (X_i\Theta - y_i)x_{i,j}$

Тогда обновление вектора $\Theta$ будет выглядеть следующим образом:
 
\begin{equation*}
   \theta_{0_{new}} = \theta_0 - \alpha  \dfrac{2}{N}\sum_{i=1}^{N} (X_i\Theta - y_i)x_{i,0}\\
   \theta_{1_{new}} = \theta_1 - \alpha  \dfrac{2}{N}\sum_{i=1}^{N} (X_i\Theta - y_i)x_{i,1}\\
   \theta_{2_{new}} = \theta_2 - \alpha  \dfrac{2}{N}\sum_{i=1}^{N} (X_i\Theta - y_i)x_{i,2}
\end{equation*}

Выполняем данную процедуру в цикле заданное число раз, либо пока наши параметры не перестанут изменяться на величину меньшую, чем некоторая нами заданная величина.

Алгоритм градиентного спуска c 3 коэффициентами можно описать следующим образом.

* Выбираем случайное значение для $\theta_0, \theta_1, \theta_2 $
* Повторить $iter$ раз:

    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $\theta_{j_{new}} = \theta_j - \alpha \dfrac{\partial Loss(\Theta)}{\partial \theta_j},
    j = 0, 1, 2$

Где $\alpha$ это коэффициент, который мы выбираем.


Наконец реализуем градиентный спуск.

In [ ]:
def gradient_descent(Theta, X, y, alpha, iters):        
    theta = Theta
    for i in range (iters):
        theta = theta - alpha * gradient_function(theta, X, y)

    return theta

In [ ]:
theta_opt = gradient_descent(Theta, X, y, 0.0001, 10000)
MSE_Loss(X, theta_opt, y)

А теперь визуализируем полученный результат.

In [ ]:
plot_new_3d_data_and_hyp_grad_des(X_room, X_lstat, y, theta_opt[0], theta_opt[1], theta_opt[2])

## Предсказание на всех данных

Мы использовали всего два признака RM и LSTAT для предсказания цен на недвижимость.  

Давайте попробуем использовать все.

In [ ]:
X_all, y = load_all_data()

In [ ]:
X_all.shape

${X}_i = \begin{pmatrix}1 & x_{i, 1} &  x_{i, 2} & x_{i, 3} & \ldots, x_{i, 13}\end{pmatrix}.$


$\Theta = \begin{pmatrix}\theta_0& \theta_1& \theta_2& \theta_3& \ldots, \theta_{13}\end{pmatrix}^T$

\begin{equation*}
\tilde{y}_i= \begin{pmatrix}x_{i, 0} & x_{i, 1}& x_{i, 2}& \ldots& x_{i, 13}\end{pmatrix} \begin{pmatrix}
    \theta_0\\
    \theta_1\\
    \theta_2\\
    \cdots\\
    \theta_{13}\\
  \end{pmatrix} =  \sum_{j=0}^{13} \theta_j x_{i, j} = X_i\Theta.
\end{equation*}

 $x_{i, 0} = 1$

In [ ]:
def add_ones(X):
    return np.hstack([np.ones(len(X)).reshape(-1,1), X])

X_all_wo = add_ones(X_all)

In [ ]:
print(X_all_wo.shape)

## Функция ошибки

Определим функцию ошибки от параметров $\Theta$ :

\begin{equation*}
Loss(\Theta) = \dfrac{1}{N}\sum_{i=1}^{N}{(\tilde{y_i} - y_i)^2}= \dfrac{1}{N} \sum_{i=1}^{N}{(X_i\Theta - y_i)^2}
\end{equation*}

Где $N$ - это количество квартир, $y_i$ - реальная цена квартиры, $\tilde{y_i}$ - предсказанная цена для $i$-oй квартиры.   

## Градиент

\begin{equation*}
   \theta_{0_{new}} = \theta_0 - \alpha  \dfrac{2}{N}\sum_{i=1}^{N} (X_i\Theta - y_i)x_{i,0}\\
   \theta_{1_{new}} = \theta_1 - \alpha  \dfrac{2}{N}\sum_{i=1}^{N} (X_i\Theta - y_i)x_{i,1}\\
   \cdot\\
   \cdot\\
   \cdot\\
   \theta_{j_{new}} = \theta_j - \alpha  \dfrac{2}{N}\sum_{i=1}^{N} (X_i\Theta - y_i)x_{i,j}\\
   \cdot\\
   \cdot\\
   \cdot\\
   \theta_{m_{new}} = \theta_m - \alpha  \dfrac{2}{N}\sum_{i=1}^{N} (X_i\Theta - y_i)x_{i,m}\\
\end{equation*}

## Градиентный спуск

Полный алгоритм градиентного спуска c $m$ коэффициентами можно описать следующим образом.

* Выбираем случайное значение для $\theta_0, \theta_1,...,\theta_m $
* Повторить $iter$ раз:

    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $\theta_{j_{new}} = \theta_j - \alpha \dfrac{\partial Loss(\Theta)}{\partial \theta_j},
    j = 0, 1, ..., m$

Где $\alpha$ это коэффициент, который мы выбираем.

In [ ]:
def gradient_function(Theta, X, y_true):
    grad = np.zeros_like(Theta)
    y_pred = linear_function(X, Theta)
    for j in range(Theta.shape[0]):       
        grad[j] = 2*np.mean((y_pred - y_true)* X[:,j])
    return grad

In [ ]:
Theta_all = np.random.random_sample((X_all.shape[1],))

In [ ]:
MSE_Loss(X_all, Theta_all, y)

In [ ]:
theta_opt = gradient_descent(Theta_all, X_all, y, 0.000001, 300)

In [ ]:
print(MSE_Loss(X_all, theta_opt, y))

# Нормализация данных

В прошлом примере у нас не получилось обучить наш пример методом градиентного спуска. Давайте попробуем понять почему.

Пусть у нас есть данные, которые были сгенерированы функцией ниже.

$y =  - 10 + x_1 + 2x_2$

Это функция вида:

$y =  \theta_0 + \theta_1x_1 + \theta_2x_2 $, 

где  $\theta_0=-10$, $\theta_1 = 1$, $\theta_2=2 $

Давайте сгенерируем данные.

In [ ]:
X1 = np.random.uniform(1, 10, 1000)
X2 = np.random.uniform(0, 1, 1000)
X = np.column_stack([X1, X2])
X = add_ones(X)

У $X1$ диапозон от 1 до 10, а у $X2$ в десять раз меньше - от 0 до 1.

In [ ]:
print(X)

In [ ]:
real_theta = np.array([-10, 1, 2])
y = linear_function(X, real_theta)

In [ ]:
plot_new_3d_data(X[:, 1], X[:, 2], y)

In [ ]:
print(MSE_Loss(X, real_theta, y))

У $X1$ диапазон от 1 до 10, а у $X2$ в десять раз меньше - от 0 до 1.

In [ ]:
theta_init = np.array([-10, 0.5, 0.5])
plot_grad(X, theta_init, y, a=0.01)

Что бы решить эту проблему нужно привести данные к одному диапазону. То есть, нормализовать данные.

Это можно решить с помощью стандартизации.

Для каждой колонки (часто ее называют фича) нужно посчитать ее среднее и стандартное отклонение (среднеквадратическое отклонение). Затем нужно из каждого элемента колонки вычесть среднее и поделить это на стандартное отклонение.

* Среднее значение (мат. ожидание): $E(X) = \dfrac{1}{N}\sum_i^NX_i$

* Дисперсия: $D(X) = \dfrac{1}{N}\sum_i^N (X_i - E(X))^2$

* Стандартное отклонение: $\sigma_X = \sqrt{D(X)}$

$X = \dfrac{X - E(X)}{\sqrt{D(X)}} = \dfrac{X - E(X)}{\sigma_X}$

In [ ]:
means = X.mean(axis=0)
stds = X.std(axis=0)

In [ ]:
print(means, stds)

In [ ]:
X_n = X.copy()

In [ ]:
for i in range(1, X_n.shape[1]):
    X_n[:, i] = (X_n[:, i] - means[i]) / stds[i]
print(X_n)

In [ ]:
plot_grad(X_n, theta_init, y, a=0.25, k_min=-3, k_max=6)

### StandardScaler

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X)
print(scaler.mean_, means)
print(scaler.scale_, stds)

Снова загрузим данные:

In [ ]:
X_all, y = load_all_data()

In [ ]:
scaler = StandardScaler()
X_all_normalize = scaler.fit_transform(X_all)

In [ ]:
X_all_normalize_with_one = add_ones(X_all_normalize)

In [ ]:
Theta = np.random.random_sample((X_all_normalize_with_one.shape[1]))
theta_opt = gradient_descent(Theta, X_all_normalize_with_one, y, 0.1, 300)
print(theta_opt)

In [ ]:
MSE_Loss(X_all_normalize_with_one, theta_opt, y)

Тогда пайплайн для обучения модели выглядит следующим образом:

1. Получить данные.
2. Стандартизировать их.
3. Добавить единицу.
4. Обучить модель.
5. Оценить результаты.

# Полиномиальная регрессия

Постановка задачи - предсказание расхода топлива в зависимости от скорости.

Данные для первой ступени ракеты, но задача может быть актуальна и для автомобилистов.

In [ ]:
X_train, y_train = get_data_for_polynom()

In [ ]:
plot_poly_data(X_train, y_train)

In [ ]:
lin_theta = np.array([0.0, 0.0])

alpha = 0.001
iters = 10000
X_wo = np.column_stack([np.ones_like(X_train), X_train])
lin_theta_opt = gradient_descent(lin_theta, X_wo, y_train, alpha, iters)

In [ ]:
visualize_prediction(X_train, y_train, linear_function(X_wo, lin_theta_opt))

Линейная регрессия хорошо работает в тех случаях, когда имеется линейная зависимость между данными.  Но что делать, если данные распределены нелинейно, если зависимость более сложная, как в нашем примере?
В данном случае линейная регрессия слишком грубо описывает данные, что приводит к большим ошибкам.

## Полиномы

Полиномом степени n называется функция:

$poly(X) = \theta_n X^n + \theta_{n-1} X^{n-1}  \ldots + \theta_1 X + \theta_0$. 

Известное вам квадратное уравнение — это полином второй степени. 

$quadratic(X) = \theta_2 X^2 + \theta_1 X + \theta_0$, или как вы привыкли его видеть: 

$quadratic(X) = a X^2 + b X + c$.

### Пример параболы

In [ ]:
plot_parabola()

### Пример полиномов больших степеней

In [ ]:
plot_polynoms()

В общем случае полином $n$-ой степени может иметь до $n$ корней, а значит пересекает ось $Х$ в $n$ точках (если $n$ корней). Для того, чтобы это было возможно он должен "колебаться" вверх-вниз. Т.е. чем выше степень полинома, тем больше таких "волн" в нем может быть.

Соответственно, такие функции могут описать более сложные зависимости данных.

## Полиномы для линейной регрессии

Давайте рассмотрим решение задачи на примере многочлена степени 5. Напомним, наша задача - нахождение расхода топлива 1-ой ступени в зависимости от скорости. Предположим, что решение будет иметь следующий вид:

$y = \theta_0 + \theta_1 x + \theta_2 x^2 + \theta_3 x^3 + \theta_4 x^4 + \theta_5 x^5$

Такая модель по-прежнему называется линейной, поскольку все коэффициенты (а т.е. веса модели) линейны. Квадраты и прочие степени появились только у $x$, т.е. у признаков модели. Получение таких признаков достигается просто возведением в нужную степень. Т.е. теперь мы будем передавать в модель не один признак $x$, a набор признаков $\begin{pmatrix} x & x^2 & x^3 & x^4 & x^5\end{pmatrix}$.

Пример:
у нас имеется набор данных:
одна строка - один пример данных

| Признак | Целевые метки |
|---------|---------------|
| $x$       | $y$             |
| 4.09    | 24            |
| 4.96    | 21.6          |
| ...     | ...           |


Что будет теперь:

| Признаки |   -   |  -  |  -  |  -   | Целевые метки |
|----------|-------|-----|-----|------|---------------|
| $x$      | $x^2$ | $x^3$ | $x^4$ | $x^5$  | $y$            |
| 4.09     | 16.73 | 68.42  | 279.83 | 1144.5 | 24            |
| 4.96     | 24.6 | 122.02 | 605.24 | 3001.98 | 21.6          |
| ...      | ...   | ... | ... | ...  | ...           |

Это до сих пор линейая регрессия потому что:

$y = \theta_0 + \theta_1 x + \theta_2 x_2 + \theta_3 x_3 + \theta_4 x_4 + \theta_5 x_5$,

где 

$x_2 = x^2$

$x_3 = x^3$

$x_4 = x^4$

$x_5 = x^5$.

То есть, у нас есть матрица $X$. 

Предсказания делаются функцией: 

$y = X\Theta$.

Матрица с данными умножается на матрицу коэффициентов.

То есть, нам не важно, что из себя представляют значения матрицы $X$. Это могут быть как просто данные тогда это обычная линейная регрессия. А если это полиномиальные значения, то получается полиномиальная регрессия.

\begin{equation*}
\mathbf{X} = \begin{pmatrix}
1 & x_0 & x_0^2 &\dots & x_0^m\\
1 & x_1 & x_1^2 &\dots & x_1^m \\
\cdots \\
1& x_N & x_N^2 &\dots & x_N^m
\end{pmatrix} = \begin{pmatrix}
1 & x_{0, 1} & x_{0, 2} &\dots & x_{0, m}\\
1 & x_{1, 1} & x_{1, 2} &\dots & x_{1, m} \\
\cdots \\
1& x_{N, 1} & x_{N, 2} &\dots & x_{N, m}
\end{pmatrix}  
\end{equation*}

## Реализуем регрессию на sklearn

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.metrics import mean_squared_error

poly_transformer = PolynomialFeatures(5)
X_poly = poly_transformer.fit_transform(X_train.reshape(-1,1))

In [ ]:
print(X_poly)

In [ ]:
scaler = StandardScaler()
X_poly_scaled = scaler.fit_transform(X_poly)
X_poly_scaled[:, 0] = 1

In [ ]:
X_poly_scaled

In [ ]:
regressor = LinearRegression().fit(X_poly_scaled, y_train)
y_pred = regressor.predict(X_poly_scaled)
print(mean_squared_error(y_train, y_pred))

In [ ]:
plot_poly_results(X_poly_scaled, y_train, poly_transformer, scaler, regressor)

Алгоритм для полиномиальной регрессии:

1. Сгенерировать полиномиальные фичи
2. Стандартизировать параметры (кроме единичной колонки)
3. Обучить линейную регрессию
4. Оценить результат

#### Переобучение

In [ ]:
interactive_polynom(X_train, y_train)

Допустим мы получили новые данные:

In [ ]:
X_test, y_test = get_more_data_for_polynom()

In [ ]:
plot_poly_data(X_train, y_train, X_test, y_test)

In [ ]:
interactive_polynom(X_train, y_train, X_test, y_test)

# Train test split

Для борьбы с переобучением набор данных делиться на два сета:
    1. Набор для тренировки.
    2. Набор для тестирования.
    
Обычно 80% идет на набор для тренировки. 

## Оценка модели

In [ ]:
interactive_polynom(X_train, y_train, X_test, y_test)

* Недообучение: большая ошибка на тренировочных и тестированных данных.
* Переобучении: маленькая ошибка на тренировочных данных и большая ошибка тестированных данных.
* Хорошо обученная модель: маленькая ошибка на тренировочных данных и маленькая ошибка тестированных данных.

# MAE

MSE не единственная функция ошибки, которая применяется для обучения регрессии.

$MAE = \frac{1}{N}\sum_{i=0}^{N}{|\hat{y_i} - y_i|}= \frac{1}{N} \sum_{i=0}^{N}{|X_i\Theta - y_i|}$

Это то, насколько в среднем ошибается наша модель.

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
X_test_poly = poly_transformer.transform(X_test.reshape(-1,1))
X_test_poly = scaler.transform(X_test_poly)
X_test_poly[:, 0] = 1
y_test_pred = regressor.predict(X_test_poly)

In [ ]:
print(mean_squared_error(y_test, y_test_pred))

In [ ]:
print(mean_absolute_error(y_train, y_pred))

### МАЕ как функция ошибки

В общем случае производная от $|x|$ не определена в точке 0, во всех остальных случаях ее можно определть, как $|x|'= \dfrac{|x|}{x}$.


При $x > 0, \ |x| = x, \ \dfrac{|x|}{x} = \dfrac{x}{x} = 1$.

При $x < 0, \ |x| = -x, \ \dfrac{|x|}{x} = \dfrac{-x}{x} = -1$.

В нашем случае мы можем доопределить производную от $|x|$ в нуле значением 0. Тогда она совпадет с функцией знака:
\begin{equation*}
 sign(x) = 
 \begin{cases}
   1 &\text{x > 0}\\
   0 &\text{x = 0}\\
   -1 &\text{x < 0}
 \end{cases}
\end{equation*}


Теперь мы можем посчитать градиент функции потерь:  
\begin{equation*}
\frac{\partial Loss(\Theta)}{\partial \theta_j} = \frac{1}{N} \sum_{i=1}^{N} sign(X_i\Theta - y_i) x_{ij}
\end{equation*}

\begin{equation*}
\nabla Loss(\Theta) = 
 \begin{bmatrix}
   \frac{1}{N} \sum_{i=1}^{N} sign(X_i\Theta - y_i)x_{i0}\\
   \frac{1}{N} \sum_{i=1}^{N} sign(X_i\Theta - y_i)x_{i1}\\
   \cdots\\
   \frac{1}{N} \sum_{i=1}^{N} sign(X_i\Theta - y_i)x_{im}\\
 \end{bmatrix}
\end{equation*}

#### В чем разница между MAE и MSE

Реакция на выбосы.

In [ ]:
 plot_outlier()

Разница в MSE возводиться в квадрат. Поэтому ошибка на выбросе будет больше учитываться.

In [ ]:
plot_regression_with_outlier()

### Скорость алгоритма

In [ ]:
plot_mae_mse()    

На большом $\alpha$ MAE не сходиться к локальному минимуму. Но при маленьком $\alpha$ MAE долго сходиться.

# Чему мы сегодня научились

* Линейная регрессия на большом количестве фич
* Стандартизация и нормализация данных
* Полиномиальная регрессия
* Переобучение, недообучение
* MAE

# Домашняя работа

Обучить полиномиальную регрессию градиентом спуском на средней абсолютной ошибке.